# Function to fit our analytical model to the SF-gas from TNG

Our fit procedure is also described in section 3 of van son et al in prep.




In [20]:
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy import constants as const

from scipy import interpolate
from scipy.optimize import minimize
from scipy.optimize import curve_fit

# from astropy.cosmology import WMAP9, z_at_value
from astropy.cosmology import Planck18  as cosmo# Planck 2018
from astropy.cosmology import z_at_value
   
base_dir    = '/Users/lieke/surfdrive/Documents/RateMassRedshiftEvolution/'
save_loc    =  base_dir+'/plots/'
TNGlocation = '/Users/lieke/surfdrive/Documents/CompareCOMPAS/'

############################
# Custom scripts
import get_ZdepSFRD as Z_SFRD
import importlib
importlib.reload(Z_SFRD)

############################
##PLOT setttings
from matplotlib import rc
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rc('font', family='serif')
rc('text', usetex=True)
fsize, SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 30,20,20,30
for obj in ['axes','xtick','ytick']:
    plt.rc(obj, labelsize=MEDIUM_SIZE)          # controls default text sizes
for obj in ['figure','axes']:
    plt.rc(obj, titlesize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize


# First read the TNG data 
## And convert it to a SFDR in Msun/yr/Mpc^-3



In [11]:

######################################
# read TNG data
######################################
with h5.File(TNGlocation+"SFRMetallicityFromGasTNG100.hdf5", "r") as f:
    MetalBins     = f["MetalBins"][:]
    Lookbacktimes = f["Lookbacktimes"][:]
    BoxSfr        = f["Sfr"][:]
# Convert SFR from sfr/box to sfr Mpc-3
littleh  = 0.6774
Rbox     = 75/littleh
TNG_SFRD = BoxSfr / Rbox**3 *u.Mpc**-3
TNG_SFRD = TNG_SFRD.value

##########################################
# "observed" TNG metallicities that we use for our calculations
##########################################
center_Zbin = (MetalBins[:-1] + MetalBins[1:])/2.
# Let's not use ALL metallicities in the TNG.. (they go waay too low!)
low_bound_Z_ind = np.where(center_Zbin > 1e-5)[0]# index of center_Zbin, where Z > 1e-5
tofit_TNG_metals = center_Zbin[low_bound_Z_ind]   


##########################################
# "obesrved" SFRD in TNG
##########################################
# flip redshifts around so they are in increasing order
TNG_SFRD_inc_z = TNG_SFRD[::-1, :]
tofit_TNG_SFRD = TNG_SFRD_inc_z[:,low_bound_Z_ind] 


##########################################
# The redshifts at which we will calc the log-skew-normal
##########################################
#### TNG redshifts corresponding to these lookback times ### 
# the last value of Lookbacktimes = 0, which is problematic for z calculation
redshifts_TNG = [z_at_value(cosmo.lookback_time,t*u.Gyr) for t in Lookbacktimes[:-1]] 
redshifts_TNG.append(0) # put redshift zero back at the end
redshifts_TNG = np.array(redshifts_TNG)
print('redshifts_TNG')  
print(['%.2f'% (z_fl) for  z_fl in redshifts_TNG[::-1] ] )
# print(np.diff(redshifts_TNG[::-1]), min(np.diff(redshifts_TNG[::-1])), max(np.diff(redshifts_TNG[::-1])))
    
reverse_lookback = Lookbacktimes[::-1]
# print('Reverse_lookback time: ')
# print(['%.2f'% (t_l) for  t_l in reverse_lookback ] )
# print(np.diff(reverse_lookback), min(np.diff(reverse_lookback)), max(np.diff(reverse_lookback)))


redshifts_TNG
['0.00', '0.01', '0.02', '0.03', '0.05', '0.06', '0.07', '0.08', '0.10', '0.11', '0.13', '0.14', '0.15', '0.17', '0.18', '0.20', '0.21', '0.23', '0.24', '0.26', '0.27', '0.30', '0.31', '0.33', '0.35', '0.36', '0.38', '0.40', '0.42', '0.44', '0.46', '0.48', '0.50', '0.52', '0.55', '0.58', '0.60', '0.62', '0.65', '0.68', '0.70', '0.73', '0.76', '0.79', '0.82', '0.85', '0.89', '0.92', '0.95', '1.00', '1.04', '1.08', '1.12', '1.16', '1.21', '1.25', '1.31', '1.36', '1.42', '1.50', '1.54', '1.61', '1.67', '1.75', '1.83', '1.91', '2.01', '2.11', '2.22', '2.33', '2.46', '2.59', '2.75', '2.91', '3.03', '3.31', '3.52', '3.74', '4.05', '4.22', '4.48', '4.72', '5.06', '5.30', '5.61', '5.94', '6.11', '6.62', '7.15', '7.40', '7.78', '8.22', '8.68', '9.28', '9.69', '10.35', '11.42', '12.53', '15.96', '22.11']


In [12]:
# What fraction of TNG metallicity happens below 10^-5
print('total SF happening in TNG', np.sum(TNG_SFRD))

higher_Z = np.where(center_Zbin > 1e-5)[0]# index of center_Zbin, where Z > 1e-5
print('Higher Z SF happening in TNG', np.sum(TNG_SFRD[:,higher_Z]))

lower_Z = np.where(center_Zbin <= 1e-5)[0]# index of center_Zbin, where Z <= 1e-5
print('Lower than 1e-5 Z SF happening in TNG', np.sum(TNG_SFRD[:,lower_Z]))

print('fraction happening below 1e-5',  np.sum(TNG_SFRD[:,lower_Z])/np.sum(TNG_SFRD) *100 ) 



total SF happening in TNG 3.6975867813633303
Higher Z SF happening in TNG 3.661207217925086
Lower than 1e-5 Z SF happening in TNG 0.0363795634382447
fraction happening below 1e-5 0.9838731472539303


# Interpolate the TNG data

Using scipy interpolate
https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html

f = interpolate.interp2d(x, y, z)



In [13]:

#########################################
x = Lookbacktimes[::-1] #redshifts_TNG[::-1]
y = tofit_TNG_metals
z = tofit_TNG_SFRD 
print('np.shape(x), np.shape(y), np.shape(z)', np.shape(x), np.shape(y), np.shape(z))

f_interp = interpolate.interp2d(x, y, z.T, kind='cubic')

# lets interpolate at regular redshift intervals
z_new    = np.arange(0, 10.1, 0.05)
# z_new = np.append(z_new, np.arange(3, 10, 0.01))
xnew     = [cosmo.lookback_time(z).value for z in z_new]
ynew     = tofit_TNG_metals #np.logspace(-5., -0.5, 100)#

SFRDnew = f_interp(xnew,ynew)

print('np.shape(xnew)', np.shape(xnew),'np.shape(ynew)', np.shape(ynew), 'np.shape(SFRDnew)', np.shape(SFRDnew) )


np.shape(x), np.shape(y), np.shape(z) (100,) (30,) (100, 30)
np.shape(xnew) (202,) np.shape(ynew) (30,) np.shape(SFRDnew) (30, 202)


In [31]:

##################################################
# muz =-0.09, mu0 =0.026, sigma =1.9, alpha=-3.3
##################################################
def calc_chi_square(fit_metals, Redshifts = redshifts_TNG[::-1],  observed_SFRD = tofit_TNG_SFRD,
                    mu_0_list = 0.026, muz_list =-0.09, sigma0_list = 1.9, sigmaz_list = 1.9, alpha_list =-3.3,
                    sf_a =0.01 , sf_b=2.6, sf_c=3.2 , sf_d=6.2, normed = False):
    """
    Calculate the distribution of metallicities at different redshifts using a log skew normal distribution
    that is basically a skew normal distribution, but then with the random variable x = ln(Z)

    NOTE: This assumes that metallicities in COMPAS are drawn from a flat in log distribution

    Args:
        fit_metals              --> [float]          metals used for fitt
        Redshifts               --> [float]          redshihts used to fit
        observed_SFRD           --> [float]          'observed' SFRD to fit to
        
        mu_0_list    = -0.23    --> [float]          location (mean in normal) at redshift 0
        muz_list = 0.035    --> [float]          redshift evolution of the location
        sigma0_list  = 0.39     --> [float]          Scale at redshift 0 (variance in normal)
        sigmaz_list  = 0.0      --> [float]          redshift evolution of Scale (variance in normal)
        alpha_list   = 0.0      --> [float]          shape (skewness, alpha = 0 retrieves normal dist)

        sf_a                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_b                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_c                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_d                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)

    Returns:
        tot_chi_square          --> [float ] 
    """
    
    """
    ######################################

    """  
    print('muz_list', muz_list, 'mu_0_list',mu_0_list, 'sigma0_list',sigma0_list,'sigmaz_list',sigmaz_list, 'alpha_list',alpha_list,
          'sf_a', sf_a, 'sf_b', sf_b, 'sf_c', sf_c, 'sf_d', sf_d)
        
    #####################################
    # Get the SFR
    # Madau & Fragos 2017: a=0.01, b=2.6, c=3.2,  d=6.2
    sfr = Z_SFRD.Madau_Dickinson2014(Redshifts, a=sf_a, b=sf_b, c=sf_c, d=sf_d) # Msun year-1 Mpc-3 

    # Get dPdZ 
    dPdlogZ, redshifts, metallicities, step_logZ, p_draw_metallicity = \
                    Z_SFRD.skew_metallicity_distribution(mu_z = muz_list, mu_0 = mu_0_list,
                                                  sigma_0= sigma0_list, sigma_z=sigmaz_list, alpha = alpha_list, 
                                                  metals=fit_metals, redsh = Redshifts)
    
    dPdZ = dPdlogZ/step_logZ

    #print('np.shape(fit_metals)', np.shape(fit_metals),'np.shape(dPdlogZ)', np.shape(dPdlogZ))
    
    ######################################
    # For each redshift in the TNG data:
    tot_chi_square = 0
    #np.zeros((len(muz_list), len(mu_0_list), len(sigma_list), len(alpha_list)) )
        
    for redshift_i in range(len(redshifts)):
        #print(redshift_i, 'at redshift', redshifts[redshift_i])
        ######################################
        # Now the SFRD = sfr x dPdZ
        #print('sfr[redshift_i]',sfr[redshift_i])
        SFRD = sfr[redshift_i] *dPdZ[redshift_i,:]  #dPdlogZ[redshift_i,:] 
        #/p_draw_metallicity# Msun year-1 Mpc-3 

        ###################
        if normed:
            obs_norm = np.sum(observed_SFRD[redshift_i,:])
            obs = observed_SFRD[redshift_i,:]/obs_norm
            #print('np.sum(obs)', np.sum(obs))
            
            #model is normed in logZ, so multiply by logZ-binwidth
            model = dPdlogZ[redshift_i,:] *step_logZ 
            #print('np.sum(model)', np.sum(model).flatten() )

        else:
            obs = observed_SFRD[redshift_i,:] 
            #print('step_logZ', step_logZ)
            model = SFRD.value 
        #print('obs', np.shape(obs) , 'model', np.shape(model) )
        
        ###################
        # Zero values are troublesome in Chi_squared!
        # only fit where obs > 1e-8  
        common_bool = np.logical_and(model > 1e-11, obs > 1e-11)
        model = model[common_bool]
        obs   = obs[common_bool]

        ###################
        # Actual CHI_squared
        #chi_square = ((np.log10(obs) - np.log10(model) )**2)#/np.sum(model) 
        chi_square = ((obs - model )**2)/np.sum(model) 
        #print('chi_square', chi_square, np.sum(chi_square, axis = -1))
        #/np.sum(model)  = divide by the sfr at this redshift
        
        #/sfr[redshift_i].value #/model 
        #print('chi_square', chi_square )
        tot_chi_square += np.sum(chi_square, axis = -1)
        #print('tot_chi_square', np.shape(tot_chi_square), tot_chi_square )


    ######################################
    # Minimum Chi_squared taking all redshift into aaccount
    print(tot_chi_square)
    
    return tot_chi_square
#     return chi_square_matix, muz_list[ind[0]], mu_0_list[ind[1]], sigma_list[ind[2]], alpha_list[ind[3]]


    
    

# Run your chi square calculations

## and leave the refinement up to scipy minimize

In [32]:
'''
default 'Neijssel et al 2019': a=0.01, b=2.77, c=2.9,  d=4.7
Madau & Fragos 2017: a=0.01, b=2.6, c=3.2,  d=6.2

z_new = np.arange(0, 10, 0.001)
xnew = [cosmo.lookback_time(z).value for z in z_new]
ynew = np.logspace(-5, -0.5, num=50)

SFRDnew = f(xnew,ynew)
'''

#################################################################
## Function wrapper to minimize the Chi_square
#################################################################
def test_chi(x0 = [-0.09, 0.026, 1.9, 0.1, -3.3,0.01, 2.6, 3.2, 6.2] ):

# Interpolated TNG data
    chi_square_matix = calc_chi_square(ynew, Redshifts = z_new, observed_SFRD = SFRDnew.T, 
                                       muz_list =x0[0], mu_0_list =x0[1],sigma0_list =x0[2], sigmaz_list=x0[3], alpha_list =x0[4],
                                       sf_a =x0[5], sf_b=x0[6], sf_c=x0[7], sf_d=x0[8],
                                       normed = False)

# Pure TNG data
#     chi_square_matix = calc_chi_square(fit_metals=tofit_TNG_metals, Redshifts = redshifts_TNG[::-1], observed_SFRD = tofit_TNG_SFRD, 
#                                        muz_list =x0[0], mu_0_list =x0[1], sigma_list =x0[2], alpha_list =x0[3],
# #                                        sf_a =0.01, sf_b=2.6, sf_c=3.2, sf_d=6.2,
#                                        sf_a =x0[4], sf_b=x0[5], sf_c=x0[6], sf_d=x0[7],
#                                        normed = False)
    
    return chi_square_matix


# BEST GUESS
x0 = np.array([-0.15, 0.026, 1.1, 0.1, -3.3, 0.01, 2.6, 3.2, 6.2])
# FIT
res = minimize(test_chi, x0= x0, method ='nelder-mead')

# BOUNDS=[(-0.3, -0.01), (0.01, 0.035), (0.5, 3.5), (-2.5, -0.1), (0.009, 0.011), (2.5,2.7), (1., 5), (1, 9)] 
# res = minimize(test_chi, x0= x0, method ='Powell', bounds=BOUNDS)

print(res.x)




muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
0.6962262296222161
muz_list -0.1575 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
0.7095124047186293
muz_list -0.15 mu_0_list 0.0273 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
0.6866431072488378
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1550000000000002 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
0.6574047924468241
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.10500000000000001 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
0.6794737481720635
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.465 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
0.6994728099619894
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.0105 sf_b 2.6 sf_c 3.2 sf_d 6.2
0.7603806377057158
muz_list -0.15 mu

0.39358210652492354
muz_list -0.14942010728076563 mu_0_list 0.02743237613955841 sigma0_list 1.2040037270969854 sigmaz_list 0.10701719609113283 alpha_list -3.3555292516329587 sf_a 0.009895889905934504 sf_b 2.1420039933366257 sf_c 3.2517948286711356 sf_d 6.0348153835158245
0.3025005078818464
muz_list -0.1523894266771532 mu_0_list 0.027020028380769567 sigma0_list 1.1946394657398542 sigmaz_list 0.10590949578889405 alpha_list -3.482892348685838 sf_a 0.010051611743211087 sf_b 2.038947273455124 sf_c 3.1555446963461167 sf_d 6.283380480628016
0.3710331154929504
muz_list -0.15403076029821122 mu_0_list 0.02753910893780652 sigma0_list 1.232020051398687 sigmaz_list 0.10902518040814493 alpha_list -3.3579523899011505 sf_a 0.00974286679918496 sf_b 2.0723213607567708 sf_c 3.1751417476696093 sf_d 6.030548054884344
0.3113074670130839
muz_list -0.15492544954872955 mu_0_list 0.027106627518359114 sigma0_list 1.2350924063643198 sigmaz_list 0.10896973126635773 alpha_list -3.376283572969418 sf_a 0.009776831291

0.19763558206752752
muz_list -0.1578998460684206 mu_0_list 0.028508928072681024 sigma0_list 1.4156223133874692 sigmaz_list 0.10993903584348792 alpha_list -3.5632041414016835 sf_a 0.009159969468755805 sf_b 1.808443730023387 sf_c 3.3640601602681137 sf_d 4.412856257725537
0.1815132800737581
muz_list -0.1544232319491184 mu_0_list 0.029061144989633192 sigma0_list 1.38126714880344 sigmaz_list 0.10635165640525926 alpha_list -3.574272538927274 sf_a 0.009074730274125378 sf_b 1.8519517257140459 sf_c 3.340623975469164 sf_d 4.60143395461863
0.1718986712872891
muz_list -0.15756843155916916 mu_0_list 0.028054532739260063 sigma0_list 1.3127788913028424 sigmaz_list 0.11173693761307509 alpha_list -3.5295489626552046 sf_a 0.009600153969546993 sf_b 1.946261641814146 sf_c 3.2398726251555354 sf_d 4.919172542766209
0.20741137225792503
muz_list -0.16181303038375086 mu_0_list 0.028071744284522644 sigma0_list 1.4904982827826383 sigmaz_list 0.11319875822352388 alpha_list -3.563367772736123 sf_a 0.00909289058988

0.09474245860255544
muz_list -0.1335058041467231 mu_0_list 0.03298716244908478 sigma0_list 1.2953199362328212 sigmaz_list 0.08759462943685059 alpha_list -4.282435250135689 sf_a 0.005711701422116557 sf_b 2.1337579828124213 sf_c 3.4336657764114538 sf_d 4.26053987175841
0.12223256116419288
muz_list -0.11889878773755208 mu_0_list 0.03363604214845208 sigma0_list 1.2342614849604292 sigmaz_list 0.08033353663418064 alpha_list -4.313414481328401 sf_a 0.005634565199201387 sf_b 2.2497103269622762 sf_c 3.260855827364936 sf_d 5.067911756119576
0.0803516888847558
muz_list -0.10371843533787661 mu_0_list 0.035114095643399304 sigma0_list 1.1872287602462008 sigmaz_list 0.06960656596242598 alpha_list -4.527351756715431 sf_a 0.004574539449222637 sf_b 2.410075571902233 sf_c 3.178287786445537 sf_d 5.451099249976517
0.09064221843784744
muz_list -0.12504589844186656 mu_0_list 0.03367493792770685 sigma0_list 1.2620346622609155 sigmaz_list 0.0853612514625543 alpha_list -4.39167793250018 sf_a 0.00525773993371301

0.07285900438756243
muz_list -0.11610647257151793 mu_0_list 0.03311563049697444 sigma0_list 1.1904099522119282 sigmaz_list 0.08181948944140433 alpha_list -4.411488835316556 sf_a 0.005563156034916708 sf_b 2.1702648499478103 sf_c 3.3668507241299244 sf_d 5.232960252302623
0.07557777481717405
muz_list -0.11713225756888906 mu_0_list 0.03365170277573871 sigma0_list 1.1980850363689077 sigmaz_list 0.08139515583591596 alpha_list -4.45797899377665 sf_a 0.005276599343185869 sf_b 2.179795715402242 sf_c 3.3613716886184446 sf_d 5.042444242989433
0.07005286208425336
muz_list -0.10951159982999215 mu_0_list 0.03469544881419507 sigma0_list 1.1352745508148578 sigmaz_list 0.07569903841460343 alpha_list -4.631993941145858 sf_a 0.004726536349079902 sf_b 2.319894604973216 sf_c 3.254668106887723 sf_d 5.229861062311549
0.07968376649083934
muz_list -0.12116232378889796 mu_0_list 0.033930065649328905 sigma0_list 1.2303446343994011 sigmaz_list 0.08294569820056658 alpha_list -4.451756934661599 sf_a 0.0051249390375

...

0.014121812582276308
muz_list -0.044048185113407315 mu_0_list 0.023975995377279145 sigma0_list 1.0967914451040812 sigmaz_list 0.04976892804455031 alpha_list -1.738791514335991 sf_a 0.006424738277681202 sf_b 1.4866698302649257 sf_c 4.450784231689571 sf_d 5.860296783567501
0.014045350755746852
muz_list -0.0415282921420118 mu_0_list 0.02397431026449065 sigma0_list 1.077847379592249 sigmaz_list 0.050437628831393244 alpha_list -1.6394197031064928 sf_a 0.0063661063257313515 sf_b 1.4989027114914473 sf_c 4.439725113220554 sf_d 5.855560940413868
0.014071460369335159
muz_list -0.042522528158074835 mu_0_list 0.023588452497503833 sigma0_list 1.0948168904785307 sigmaz_list 0.0504763044334766 alpha_list -1.7581749166423974 sf_a 0.006310624459679365 sf_b 1.5142068316259856 sf_c 4.406670300376902 sf_d 5.844944841936744
0.01409796465249867
muz_list -0.04270242601988844 mu_0_list 0.023974133311850478 sigma0_list 1.0898488563680133 sigmaz_list 0.05008158516725178 alpha_list -1.698169620241286 sf_a 0.0064

0.013980772415267746
muz_list -0.04625942584490092 mu_0_list 0.024398216320488053 sigma0_list 1.1032707097495296 sigmaz_list 0.04860504870366297 alpha_list -1.7067592154372488 sf_a 0.006452980120569853 sf_b 1.485413393525346 sf_c 4.454143606193057 sf_d 5.909089145391926
0.01396450900080206
muz_list -0.0467247795858019 mu_0_list 0.024538628634807602 sigma0_list 1.1023104870859948 sigmaz_list 0.04859096990577483 alpha_list -1.692346845423634 sf_a 0.00643141612935658 sf_b 1.4923996786422542 sf_c 4.449483712776086 sf_d 5.925401484358751
0.0139774205706419
muz_list -0.04771504238823217 mu_0_list 0.024311402866783627 sigma0_list 1.1184966293192233 sigmaz_list 0.047579429297487805 alpha_list -1.7748183742218426 sf_a 0.006547544804429836 sf_b 1.4673347922204285 sf_c 4.464295346235925 sf_d 5.91331759475664
0.013988062491149516
muz_list -0.048867309044187 mu_0_list 0.02444900092687291 sigma0_list 1.1190921349335725 sigmaz_list 0.047402161375313306 alpha_list -1.7692222739953483 sf_a 0.0064863160

0.013937603336482824
muz_list -0.04809253131203761 mu_0_list 0.02458996059647261 sigma0_list 1.1057283041492645 sigmaz_list 0.048797331902541 alpha_list -1.7257778677463165 sf_a 0.006484921412118452 sf_b 1.4746027424067782 sf_c 4.4633086670622735 sf_d 5.912571383019669
0.013942894546348814
muz_list -0.048809929598265855 mu_0_list 0.024775575892992452 sigma0_list 1.111124302090385 sigmaz_list 0.04802048763058314 alpha_list -1.7105356500723525 sf_a 0.006489280595182782 sf_b 1.4829677343463485 sf_c 4.451552129784563 sf_d 5.91654713519379
0.013949003648509954
muz_list -0.04791071002631371 mu_0_list 0.024503308002180375 sigma0_list 1.113345894354572 sigmaz_list 0.04815248610369714 alpha_list -1.7367348313311322 sf_a 0.006494556143404594 sf_b 1.4738886777157556 sf_c 4.463697622397285 sf_d 5.914011923622109
0.013937520396020869
muz_list -0.0480542936566926 mu_0_list 0.024646694299935823 sigma0_list 1.1094560076145865 sigmaz_list 0.04831280268791416 alpha_list -1.7190696513426125 sf_a 0.006455

0.013923963368062665
muz_list -0.04925527471903439 mu_0_list 0.024723839481797426 sigma0_list 1.1248004711729527 sigmaz_list 0.047916228480840664 alpha_list -1.7780426369088476 sf_a 0.006485364085350545 sf_b 1.478994087925058 sf_c 4.45089558791772 sf_d 5.891309529843147
0.013924017381835475
muz_list -0.050058734861397565 mu_0_list 0.02478971189430729 sigma0_list 1.1320562564507977 sigmaz_list 0.04763630984984688 alpha_list -1.8001098421990513 sf_a 0.006481489025777403 sf_b 1.4848178462196566 sf_c 4.43826414238343 sf_d 5.885113800029133
0.013926767123289404
muz_list -0.049487575625611124 mu_0_list 0.02466256812869652 sigma0_list 1.1290886432799065 sigmaz_list 0.04763256762681224 alpha_list -1.7834609302044013 sf_a 0.0064661828519592505 sf_b 1.4829711184089036 sf_c 4.449265601866447 sf_d 5.902202004917363
0.013925220459604004
muz_list -0.04846130776541458 mu_0_list 0.024616455812239414 sigma0_list 1.1164282445069624 sigmaz_list 0.04799510123297391 alpha_list -1.7399825472095252 sf_a 0.00

0.013919664155428566
muz_list -0.049047912596253865 mu_0_list 0.02467797181235941 sigma0_list 1.1276424742112476 sigmaz_list 0.04791540079324869 alpha_list -1.7801175537784815 sf_a 0.006499672434606411 sf_b 1.4791394728268719 sf_c 4.451470569420436 sf_d 5.903409217749287
0.013918706655944616
muz_list -0.048909648806232414 mu_0_list 0.02465028094099382 sigma0_list 1.127465546952013 sigmaz_list 0.047993584042275286 alpha_list -1.77899975830059 sf_a 0.006498457060430583 sf_b 1.479147603550568 sf_c 4.4525246927191215 sf_d 5.907216637419225
0.01391935958847441
muz_list -0.04940085563378108 mu_0_list 0.02474320563818188 sigma0_list 1.130260519540894 sigmaz_list 0.04764743994302108 alpha_list -1.7820686510592036 sf_a 0.006523319077373665 sf_b 1.4744457535221107 sf_c 4.457288770262251 sf_d 5.909009847752341
0.013919358372613592
muz_list -0.04936003627922768 mu_0_list 0.02476072672188332 sigma0_list 1.1294482229953198 sigmaz_list 0.04765194073072903 alpha_list -1.7757627366503286 sf_a 0.0065003

In [33]:
print(res.success, res.message)
muz_best, mu0_best, sigma0_best, sigmaz_best, alpha_best = res.x[0], res.x[1], res.x[2], res.x[3],res.x[4]
sf_a_best, sf_b_best, sf_c_best, sf_d_best    = res.x[5], res.x[6], res.x[7], res.x[8] #0.01, 2.6, 3.2, 6.2#

print('\nBEST FITTING PARAMETERS:')
print('muz =%s, mu0 =%s, sigma_0 =%s, sigma_z =%s, alpha=%s'% (muz_best, mu0_best, sigma0_best, sigmaz_best, alpha_best) )
print('sf_a =%s, sf_b =%s, sf_c =%s, sf_d =%s'% (sf_a_best, sf_b_best, sf_c_best, sf_d_best) )


False Maximum number of function evaluations has been exceeded.

BEST FITTING PARAMETERS:
muz =-0.04894847240898909, mu0 =0.024715336491911655, sigma_0 =1.126549927619688, sigma_z =0.04795620843423471, alpha=-1.7734788599284306
sf_a =0.006488790382531332, sf_b =1.4810631763436888, sf_c =4.451590753663159, sf_d =5.906473128991022
